In [1]:
from basic_pitch.inference import predict
import pretty_midi
from basic_pitch import ICASSP_2022_MODEL_PATH
from IPython.display import Audio

In [2]:
model_output, midi_data, note_events = predict("data/forest.m4a")


Predicting MIDI for data/forest.m4a...


C:\Users\user\.virtualenvs\KIDS\lib\site-packages\basic_pitch\inference.py:89: UserWarning: PySoundFile failed. Trying audioread instead.
  audio_original, _ = librosa.load(str(audio_path), sr=AUDIO_SAMPLE_RATE, mono=True)
C:\Users\user\.virtualenvs\KIDS\lib\site-packages\librosa\core\audio.py:183: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


In [3]:
def midi_to_notes(midi_data):
    notes = []
    beats_per_second = midi_data.estimate_tempo() / 60
    avg_note_time = 1 / beats_per_second
    print(avg_note_time)
    for instrument in midi_data.instruments:
        for i, note in enumerate(instrument.notes):
            note_time = note.end - note.start
            if note_time / avg_note_time < 0.1 \
                and i > 0:
                print(note_time / avg_note_time)

                print(note.pitch)
                if note.pitch == instrument.notes[i-1].pitch:
                    instrument.notes[i-1].end = note.end
                    continue

            notes.append(note)
            last_note = note

    return notes

In [4]:
notes = midi_to_notes(midi_data)
notes


0.4008651927437641


[Note(start=0.661769, end=1.091338, pitch=60, velocity=93),
 Note(start=1.102948, end=1.509297, pitch=69, velocity=95),
 Note(start=1.509297, end=1.892426, pitch=69, velocity=93),
 Note(start=1.904036, end=2.242010, pitch=67, velocity=86),
 Note(start=2.288449, end=2.683189, pitch=69, velocity=89),
 Note(start=2.683189, end=3.066318, pitch=65, velocity=89),
 Note(start=3.101148, end=3.507497, pitch=60, velocity=91),
 Note(start=3.507497, end=3.913846, pitch=60, velocity=94),
 Note(start=3.913846, end=4.286649, pitch=60, velocity=89),
 Note(start=4.309869, end=4.855538, pitch=69, velocity=95),
 Note(start=4.855538, end=5.099348, pitch=69, velocity=84),
 Note(start=5.134178, end=5.494087, pitch=70, velocity=91),
 Note(start=5.505697, end=5.865606, pitch=74, velocity=88),
 Note(start=5.958486, end=6.876958, pitch=72, velocity=94),
 Note(start=6.760859, end=6.946618, pitch=67, velocity=54),
 Note(start=7.039498, end=7.445847, pitch=72, velocity=92),
 Note(start=7.469067, end=7.875416, pitc

In [10]:
import numpy as np
from bokeh.layouts import layout
from bokeh.models import Div
from bokeh.plotting import figure, show

# create a plot demonstrating a sine wave
p1 = figure(height=300)

p1.xaxis.fixed_location = 0

# create a plot demonstrating a cosine wave
p2 = figure(height=300)

for note in notes:
    p1.line((note.start, note.end), note.pitch, line_width=7, line_color="darkgreen")
    
for i, note in enumerate(notes):
    if i % 3 == 0:
        p2.line((note.start, note.end), note.pitch + i%6 + 1, line_width=7, line_color="red")
        continue
    p2.line((note.start, note.end), note.pitch, line_width=7, line_color="darkgreen")

p2.xaxis.fixed_location = 0

# define the layout
layout = layout(
    [
        p1,
        p2,
    ],
    sizing_mode="stretch_width",
)

show(layout)

In [6]:
for note in notes:
    time = note.end - note.start
    print(f"start: {note.start} end: {note.end}")
    print(pretty_midi.note_number_to_name(note.pitch), note.pitch)
    print("-"*15)

start: 0.6617687074829932 end: 1.0913378684807256
C4 60
---------------
start: 1.1029478458049886 end: 1.509297052154195
A4 69
---------------
start: 1.509297052154195 end: 1.8924263038548752
A4 69
---------------
start: 1.9040362811791383 end: 2.2420095238095237
G4 67
---------------
start: 2.288449433106576 end: 2.6831886621315193
A4 69
---------------
start: 2.6831886621315193 end: 3.0663179138321994
F4 65
---------------
start: 3.1011478458049884 end: 3.507497052154195
C4 60
---------------
start: 3.507497052154195 end: 3.9138462585034013
C4 60
---------------
start: 3.9138462585034013 end: 4.286649433106576
C4 60
---------------
start: 4.309869387755102 end: 4.855538321995465
A4 69
---------------
start: 4.855538321995465 end: 5.099347845804989
A4 69
---------------
start: 5.1341777777777775 end: 5.494087074829932
A#4 70
---------------
start: 5.505697052154195 end: 5.865606349206349
D5 74
---------------
start: 5.958486167800453 end: 6.876958276643991
C5 72
---------------
start:

In [7]:
note = midi_data.instruments[0].notes[0]
pretty_midi.note_number_to_name(note.pitch)

'C4'